In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [4]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available



In [7]:

if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)


In [9]:
amazon_df

,title,price,rating,reviews,availability
8,OUBANG 2 Pack Controllers Work with PS4 Contro...,$37.99,4.8 out of 5 stars,8 ratings,Not Available
9,PlayStation 4 Slim 1TB Console,,4.7 out of 5 stars,"15,164 ratings",
10,PlayStation 4 Slim 1TB Console - Only On PlayS...,,4.8 out of 5 stars,"8,860 ratings",Only 3 left in stock - order soon
11,Hunting Simulator - PlayStation 4,$16.61,4.3 out of 5 stars,937 ratings,Not Available
12,BlueFire Professional 3.5mm PS4 Gaming Headset...,,4.4 out of 5 stars,"8,257 ratings",Not Available
13,Horizon Zero Dawn Complete Edition Hits - Play...,$22.45,4.8 out of 5 stars,"10,127 ratings",Not Available
14,"Beastron PS4 Controller Charging Station, PS4 ...",,4.4 out of 5 stars,568 ratings,Currently unavailable.
15,PlayStation 4 Slim 1TB Limited Edition Console...,,4.6 out of 5 stars,711 ratings,
16,PlayStation 4 500GB Console [Old Model][Discon...,,4.6 out of 5 stars,"13,569 ratings",
17,Sony Playstation PS4 1TB Black Console,$619.00,4.7 out of 5 stars,853 ratings,Not Available
